In [39]:
from openai import OpenAI
import os
from key import get_key

In [40]:
client = OpenAI(
    api_key= get_key()
)

In [41]:
layout_folder = "../layouts/new"

In [42]:
""" with open("../layouts/new/impostazioni.xml", "r") as file:
    layout_xml = file.read()



# Chiamata al modello GPT
response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt,}], 
        model="gpt-4o-mini"
    )

start_seq = "```python"
end_seq = "```"

# Estrai il contenuto dal response
content = response.choices[0].message.content

# Trova la posizione del primo e dell'ultimo delimitatore
start_index = content.find(start_seq) + len(start_seq)
end_index = content.rfind(end_seq)  # Usa rfind per trovare l'ultimo delimitatore

# Estrai il codice tra i delimitatori
code = content[start_index:end_index].strip()

# Salva i test generati in un file
output_file = f"../tests/generated_tests_impostazioni.py"
with open(output_file, "w") as output:
    output.write(code) """

' with open("../layouts/new/impostazioni.xml", "r") as file:\n    layout_xml = file.read()\n\n\n\n# Chiamata al modello GPT\nresponse = client.chat.completions.create(\n        messages=[{"role": "user", "content": prompt,}], \n        model="gpt-4o-mini"\n    )\n\nstart_seq = "```python"\nend_seq = "```"\n\n# Estrai il contenuto dal response\ncontent = response.choices[0].message.content\n\n# Trova la posizione del primo e dell\'ultimo delimitatore\nstart_index = content.find(start_seq) + len(start_seq)\nend_index = content.rfind(end_seq)  # Usa rfind per trovare l\'ultimo delimitatore\n\n# Estrai il codice tra i delimitatori\ncode = content[start_index:end_index].strip()\n\n# Salva i test generati in un file\noutput_file = f"../tests/generated_tests_impostazioni.py"\nwith open(output_file, "w") as output:\n    output.write(code) '

In [43]:
for filename in os.listdir(layout_folder):
    if filename.endswith(".xml"):
        file_path = os.path.join(layout_folder, filename)

        with open(file_path, "r") as file:
            layout_xml = file.read()

        prompt = f"""
            Given the following XML file representing the layout of an Android application called "ThunderBird":
            {layout_xml}

            Generate automated GUI test cases in Python using the Appium framework.
            Ensure the tests:
            1. Verify the presence of UI components.
            2. Test basic interactions like button clicks, text entry, etc.
            3. Validate navigation between screens.

            Please ensure that all comments are correctly formatted in Python syntax with the '#' symbol at the beginning
            of each comment. Do not generate any comments without the '#' symbol.
        """

        # Chiamata al modello GPT
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt, }],
            model="gpt-4o-mini"
        )
        
        start_seq = "```python"
        end_seq = "```"
        
        # Estrai il contenuto dal response
        content = response.choices[0].message.content
        # Trova la posizione del primo e dell'ultimo delimitatore
        start_index = content.find(start_seq) + len(start_seq)
        # Usa rfind per trovare l'ultimo delimitatore
        end_index = content.rfind(end_seq)
        # Estrai il codice tra i delimitatori
        code = content[start_index:end_index].strip()
        
        # Salva i test generati in un file
        output_file = f"../tests/generated_tests_{os.path.splitext(filename)[0]}.py"
        with open(output_file, "w") as output:
            output.write(code)